In [1]:
from datetime import datetime as dt
import json, re, os, sys, io, copy, socket, base64, traceback, time, pytz

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql import DataFrame, Row, Column
from delta.tables import *

In [2]:
df1 = spark.createDataFrame([Row(id = 1)])
df1.show()

+---+
 id|
+---+
 1|
+---+

In [3]:
df2 = df1.select(df1.id.astype("long"), 
                   F.lit(None).astype("long").alias("some_int"),
                   F.lit("ok").astype("string").alias("some_string"),
                   F.lit(None).astype("boolean").alias("some_bool"),
                   F.lit(None).astype("double").alias("some_float")
                  )

In [4]:
display(df2)

id,some_int,some_string,some_bool,some_float
1,null,ok,null,null


In [5]:
df2_test = df2.toJSON().collect()

In [6]:
print(df2_test)

['{"id":1,"some_string":"ok"}']

##### Fields added as `F.lit(None)` show up as `null` BUT not recognized as NULL!!

In [8]:
display(df2.fillna("LLUN"))

id,some_int,some_string,some_bool,some_float
1,null,ok,null,null


In [9]:
df2.write.format("delta").mode("append").save('/mnt/common/test_nulls')

In [10]:
df02 = spark.createDataFrame([Row(id = 2, some_int = 1, some_float = 1.1)])

In [11]:
df02.write.format("delta").mode("append").save('/mnt/common/test_nulls')

In [12]:
dd1 = spark.sql("""
SELECT * FROM delta.`/mnt/common/test_nulls`
""")
display(dd1)

id,some_int,some_string,some_bool,some_float
1,null,ok,null,null
2,1,null,null,1.1


In [13]:
dd1.printSchema()

root
-- id: long (nullable = true)
-- some_int: long (nullable = true)
-- some_string: string (nullable = true)
-- some_bool: boolean (nullable = true)
-- some_float: double (nullable = true)

##### *some_bool* was not provided when writing to disk, so written as `null` BUT not recognized as NULL!!

In [15]:
display(dd1.fillna("LLUN"))

id,some_int,some_string,some_bool,some_float
1,null,ok,null,null
2,1,LLUN,null,1.1


In [16]:
dbutils.fs.rm('/mnt/common/test_nulls', True)

Out[43]: True